In [2]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
import pandas as pd
import numpy as np
import resnet
import os
from tqdm import tqdm_notebook
from keras.preprocessing.image import load_img
from keras.models import load_model
print("Done!")

Using TensorFlow backend.


Done!


In [2]:
train_df=pd.read_csv(os.getcwd()+'/train.csv')
test_df=pd.read_csv(os.getcwd()+'/val.csv')

In [3]:
train_df["data"] = [np.array(load_img(os.getcwd()+"/train/{}".format(idx), target_size=(122,122))) / 255 for idx in tqdm_notebook(train_df['images'])]

A Jupyter Widget

In [4]:
test_df["data"] = [np.array(load_img(os.getcwd()+"/train/{}".format(idx), target_size=(122,122))) / 255 for idx in tqdm_notebook(test_df['images'])]

A Jupyter Widget

In [24]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)

batch_size = 32
nb_classes = 2
nb_epoch = 200
data_augmentation = False
csv_logger = CSVLogger('resnet18_cat_dog.csv')
# input image dimensions
img_rows, img_cols = 122, 122
# The CIFAR10 images are RGB.
img_channels = 3

In [9]:
y_train=np_utils.to_categorical(train_df['label'], nb_classes)

In [11]:
y_test=np_utils.to_categorical(test_df['label'], nb_classes)

In [15]:
X_train = np.array(train_df['data'].tolist())
X_test = np.array(test_df['data'].tolist())

In [22]:
model = resnet.ResnetBuilder.build_resnet_34((img_channels, img_rows, img_cols), nb_classes)

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
model_checkpoint = ModelCheckpoint("./resnet_cat_dog.model", save_best_only=True, verbose=1)
model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_data=(X_test, y_test),
              shuffle=True,
              callbacks=[lr_reducer, early_stopper, csv_logger,model_checkpoint])


Train on 8000 samples, validate on 2000 samples
Epoch 1/200
8000/8000 [==============================] - 28s 4ms/step - loss: 0.5107 - acc: 0.8356 - val_loss: 1.3173 - val_acc: 0.5510

Epoch 00001: val_loss improved from inf to 1.31728, saving model to ./resnet_cat_dog.model
Epoch 2/200
8000/8000 [==============================] - 28s 4ms/step - loss: 0.4885 - acc: 0.8512 - val_loss: 0.7159 - val_acc: 0.7435

Epoch 00002: val_loss improved from 1.31728 to 0.71587, saving model to ./resnet_cat_dog.model
Epoch 3/200
8000/8000 [==============================] - 28s 4ms/step - loss: 0.4399 - acc: 0.8649 - val_loss: 0.8397 - val_acc: 0.6305

Epoch 00003: val_loss did not improve from 0.71587
Epoch 4/200
8000/8000 [==============================] - 28s 4ms/step - loss: 0.4170 - acc: 0.8739 - val_loss: 5.1765 - val_acc: 0.5165

Epoch 00004: val_loss did not improve from 0.71587
Epoch 5/200
8000/8000 [==============================] - 28s 4ms/step - loss: 0.3847 - acc: 0.8894 - val_loss: 0.609

In [9]:
model=load_model("./resnet_cat_dog.model")

In [4]:
test_y=pd.read_csv(os.getcwd()+'/test.csv')
test=pd.DataFrame()
test['label']=test_y['label']
test['images']=test_y['images']


        

In [5]:
print(len(test['images']))

8000


In [6]:
test["data"] = [np.array(load_img(os.getcwd()+"/train/{}".format(idx), target_size=(122,122))) / 255 for idx in tqdm_notebook(test['images'])]

A Jupyter Widget

In [7]:
test_in=np.array(test['data'].tolist())
test_in.shape

(8000, 122, 122, 3)

In [10]:

pre=model.predict(test_in)

In [75]:
pre.shape

(5000, 2)

In [72]:
for i in range(pre.shape[0]):
    if pre[i][0] < pre[i][1]:
        pre[i][0]=0
        pre[i][1]=1
    else:
        pre[i][1]=0
        pre[i][0]=1        

In [60]:
test_re=np_utils.to_categorical(test['label'], nb_classes)

0.0


In [96]:
correct=0
wrong=0
for i in range(pre.shape[0]):
    if not np.all(pre[i][0]==test_in[i][0]):
        correct+=1
    else:
        wrong+=1
print("Correct:",correct)
print("Wrong:",wrong)
print("Acc:",float(correct/(correct+wrong)))

Correct: 4992
Wrong: 8
Acc: 0.9984
